- 라인트래킹O
- 객체 인식O
- 단, 인식되는 객체가 많으면 라인트래킹이 잘 안 됨.
- 객체 인식했을 때 정지함.

In [1]:
import threading
from socket import *
import time
from pop import Pilot

cam = Pilot.Camera(width=300, height=300)
ac = Pilot.AutoCar()
OF = Pilot.Object_Follow(cam)
LF = Pilot.Track_Follow(camera=cam)
CA = Pilot.Collision_Avoid(cam)

# 저장된 모델 3개를 불러오기
LF.load_model("line_tracer_0313_2.pth") # 라인트래킹 모델 로드
OF.load_model() # 객체 인식 모델 불러오기 ("ssd_mobilenet_v2_coco.engine"를 자동으로 불러옴)
CA.load_model("collision_final.pth") # 장애물 회피 모델 로드

LF.show()
OF.show()
CA.show()

LF.run()
CA.run()

steering_correction = 0.05
ac.setSpeed(0)

Load completed.


Image(value=b'', format='jpeg', height='300', width='300')

Image(value=b'', format='jpeg', height='300', width='300')

In [2]:
# RC카 서버의 IP 주소와 포트 번호
host = ''
port = 9851

block = 0
check = 0
destination = None
arrived = None
# arrived_lock = threading.Lock()  # arrived 변수를 동기화하는 데 사용할 Lock 객체
dest = {1: '집', 2: '회사', 3: '학교'}

# 객체 감지 쓰레드와 라인 트래킹 쓰레드를 전역 변수로 선언
object_detection_thread = None
line_tracking_thread = None

def drive(value, value2):   # value는 라인 트래킹 value값, value2는 장애물 회피 value값
    global arrived, check
    
    if check == 1:  # 주행을 수행
        if arrived is not None:  # 목적지에 도착했을 때
            print(f"{dest[arrived]}로 이동 완료")
            ac.stop()
            while arrived:  # 목적지에 도착한 상태에서 클라이언트가 다시 명령을 내지 않으면 대기
                time.sleep(1)
        else:  # 목적지에 도착하지 않았을 때
            print("----------drive: 차량 주행 코드----------")
            ac.setSpeed(10)
            ac.forward()
            steer = value['x']
            
            # 라인트랙킹을 위한 좌우 조정 (value를 사용)
            if steer > 1:
                steer = 1
            elif steer < -1:
                steer = -1
            
            ac.steering = steer * 1.5
            
            # 장애물 인식 후 후진하게 하는 코드(value2를 사용)
            if value2 >= 0.5:    # 0.5 초과 시 후진
                ac.backward()

def object_detection():
    global arrived
    print("----------Object Detection----------")
    try:
        while True:
            # 객체 감지를 수행하는 로직
            print("객체 감지 수행 중...")
            v = OF.detect(index='chair') 
            v2 = OF.detect(index='person')
            v3 = OF.detect(index='umbrella')
            
            # 객체 감지 주기 조정
            time.sleep(1)  # 예시로 0.5초로 설정
            
            if v is not None or v2 is not None or v3 is not None: 
                size_rates = [obj.get('size_rate', 0) if obj is not None else 0 for obj in [v, v2, v3]]
                print("size_rates: ", size_rates)

                if size_rates:  
                    max_size_rate = max(size_rates)
                    print("max_size_rate: ", max_size_rate)

                    if max_size_rate >= 0.2:
                        if max_size_rate == v.get('size_rate', 0) if v is not None else 0:
                            print("의자 감지", max_size_rate)
                            arrived = 1
                        elif max_size_rate == v2.get('size_rate', 0) if v2 is not None else 0:
                            print("사람 감지", max_size_rate)
                            arrived = 2
                        elif max_size_rate == v3.get('size_rate', 0) if v3 is not None else 0:
                            print("우산 감지", max_size_rate)
                            arrived = 3
                else:
                    print("감지된 객체가 없습니다.")
    except KeyboardInterrupt:
        pass

def line_tracking():
    print("----------Line Tracking----------")
    try:
        while True:
            # 라인 트래킹을 수행하는 로직
            print("라인 트래킹 수행 중...")
            
            value = LF.run()
            value2 = CA.run()
            drive(value, value2)

            if check == -1 or arrived:
                ac.stop()
                print("-----------------주행 종료-----------------")
                break
            
    except KeyboardInterrupt:
        pass

def cardrive():
    global arrived
    print("----------Cardrive: Autonomous Driving----------")
    try:
        while check != -1:  # check가 -1이 아닐 때만 실행
            if check == 1:
                # 객체 감지 및 라인 트래킹을 동시에 수행하는 로직
                object_detection_thread = threading.Thread(target=object_detection)
                line_tracking_thread = threading.Thread(target=line_tracking)

                object_detection_thread.start()
                line_tracking_thread.start()

                object_detection_thread.join()  # 쓰레드가 종료될 때까지 기다립니다.
                line_tracking_thread.join()  # 쓰레드가 종료될 때까지 기다립니다.

                time.sleep(0.1)  # 예시로 0.1초에 한 번씩 주요 작업 수행
    except KeyboardInterrupt:
        pass
    finally:
        # 예외가 발생했을 때 쓰레드를 종료합니다.
        print("예외 발생: 쓰레드 종료")
        if object_detection_thread.is_alive():
            object_detection_thread.join()
        if line_tracking_thread.is_alive():
            line_tracking_thread.join()



def send(sock):
    global check, block, arrived, dest
    carStatus = {'종료': 0, '시작': 4, '정지': 5, '장애물 감지': 6, '정상 주행 중': 7}
    
    while True:
        if check == 0:
            print("정지 중")
            time.sleep(1)
            sendData = carStatus["정지"]
        elif check == 1 and block == 2:
            print("장애물 감지")
            time.sleep(3)
            sendData = carStatus["장애물 감지"]
            sock.send(sendData.to_bytes(4, byteorder="big"))
        elif check == 1 and block == 0:
            if arrived is not None:
                print(f"{dest[arrived]}로 이동 완료")
                sendData = arrived
                sock.send(sendData.to_bytes(4, byteorder="big"))
                arrived = None
                time.sleep(3)
                check = 0
            else:
                time.sleep(10)
                sendData = carStatus["정상 주행 중"]
                sock.send(sendData.to_bytes(4, byteorder="big"))

def receive(sock):
    global check
    while True:
        data = sock.recv(1024)
        command = data.decode("utf-8").strip()
        print(command)
        if command == "시작":
            check = 1
            print("주행을 시작합니다.")
        elif command in dest.values():
            check = 1
            destination = 1 if command == '집' else (2 if command == '회사' else (3 if command == '학교' else None))
            print(f"{command}으로 이동합니다.")
        elif command == "종료":
            check = -1
            print("주행을 종료합니다.")
            break
        elif command == "정지":
            print("정지합니다.")
            ac.setSpeed(0)
            check = 0
        elif command.isdigit():
            speed = int(command)
            ac.setSpeed(speed)
            print("속도", speed)

def handle_client(client_sock):
    try:
        sender = threading.Thread(target=send, args=(client_sock,))
        receiver = threading.Thread(target=receive, args=(client_sock,))
        driver = threading.Thread(target=cardrive)

        sender.start()
        receiver.start()

        cnt = 0
        while True:
            time.sleep(1)
            if cnt == 0:
                print("차량 주행 시작")
                driver.start()
                cnt = 1
            if check == -1:
                ac.stop()  # 수정: Car -> ac
                print("-----------------정지-----------------")
                cnt = 0
                break
    finally:
        # 클라이언트 소켓이 닫히면 해당 IP와의 소켓 연결이 정상적으로 해제됨을 알림
        print("연결이 종료되었습니다:", client_sock.getpeername())
        client_sock.close()

def accept_clients(server_sock):
    while True:
        client_sock, addr = server_sock.accept()
        print(addr, "와 연결완료")
        threading.Thread(target=handle_client, args=(client_sock,)).start()


In [3]:
try:
    server_sock = socket(AF_INET, SOCK_STREAM)
    server_sock.bind((host, port))
    server_sock.listen(10)
    print("연결을 기다리는 중")
    accept_clients(server_sock)

except OSError as e:
    print(e)
    print("연결 오류")
    server_sock.close()
except Exception as e:
    print(e)
finally:
    server_sock.close()

연결을 기다리는 중
('192.168.55.100', 59314) 와 연결완료
정지 중
차량 주행 시작정지 중
----------Cardrive: Autonomous Driving----------

정지 중
정지 중
정지 중
정지 중
정지 중
정지 중
정지 중
정지 중
정지 중
회사
회사으로 이동합니다.
----------Object Detection----------
----------Line Tracking----------객체 감지 수행 중...

라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
객체 감지 수행 중...
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
객체 감지 수행 중...
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
객체 감지 수행 중...
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
객체 감지 수행 중...
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
객체 감지 수행 중...
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
객체 감지 수행 중...
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
객체 감지 수행 중...
라인 트래킹 수행 중...
----------drive: 차량 주행 코드----------
라인 트래킹 수행 중...
----------drive: 차

KeyboardInterrupt: 